In [43]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [44]:
import argparse
from math import log10

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from data import DatasetFromFolder
from model import SRCNN
import csv


In [45]:
SRCNN().eval()

SRCNN(
  (conv1): Conv2d(1, 64, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
  (conv2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
  (conv3): Conv2d(32, 1, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
)

In [46]:
zoom_factor = 2
nb_epochs = 50
cuda = True


In [47]:
from datetime import datetime
start_time = datetime.now()

device = torch.device("cuda:0" if (torch.cuda.is_available() and cuda) else "cpu")
torch.manual_seed(0)
torch.cuda.manual_seed(0)

# Parameters
BATCH_SIZE = 8
NUM_WORKERS = 0 # on Windows, set this variable to 0

trainset = DatasetFromFolder("train", zoom_factor=zoom_factor)


trainloader = DataLoader(dataset=trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)


model = SRCNN().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(  # we use Adam instead of SGD like in the paper, because it's faster
    [
        {"params": model.conv1.parameters(), "lr": 0.0001},  
        {"params": model.conv2.parameters(), "lr": 0.0001},
        {"params": model.conv3.parameters(), "lr": 0.0001},
    ], lr=0.0001,
)

idx = 0
cols = ['Nomor','Epoch','Loss']
data_rows = []

for epoch in range(nb_epochs):

    # Train
    epoch_loss = 0
    for iteration, batch in enumerate(trainloader):
        input, target = batch[0].to(device), batch[1].to(device)
        optimizer.zero_grad()

        out = model(input)
        loss = criterion(out, target)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    

    print(f"Epoch {epoch}. Training loss: {epoch_loss / len(trainloader)}")
    idx = idx + 1
    data_rows.append([idx, epoch, epoch_loss / len(trainloader)])
    #filewriter.writerow([idx,epoch,epoch_loss / len(trainloader)])
    # Save model
    # torch.save(model, f"model/model_{epoch}.pth")
    torch.save({'epoch': epoch,
                'model': model,
                'optimizer': optimizer},
                'model/Epoch 500 Bt 8 Upscale x8_SRCNN.pth.tar')

with open('model/4.csv','w') as csvfile:
    c = csv.writer(csvfile, delimiter = ',', quotechar = '|', quoting = csv.QUOTE_MINIMAL, dialect = 'excel')
    c.writerow(cols);
    c.writerows(data_rows)

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Epoch 0. Training loss: 0.010855228337703493
Epoch 1. Training loss: 0.0011699555514598512
Epoch 2. Training loss: 0.0005809610934542994
Epoch 3. Training loss: 0.0004464222672452395
Epoch 4. Training loss: 0.00037478774332545594
Epoch 5. Training loss: 0.0003249728540324938
Epoch 6. Training loss: 0.0002879461892160027
Epoch 7. Training loss: 0.0002601182854928115
Epoch 8. Training loss: 0.0002394849191793449
Epoch 9. Training loss: 0.00022379309234088872
Epoch 10. Training loss: 0.00021107588487415138
Epoch 11. Training loss: 0.00020124825042565808
Epoch 12. Training loss: 0.00019326266584147772
Epoch 13. Training loss: 0.00018671039331413748
Epoch 14. Training loss: 0.00018082202309510552
Epoch 15. Training loss: 0.00017558380278495753
Epoch 16. Training loss: 0.0001713881008892802
Epoch 17. Training loss: 0.0001682727658710853
Epoch 18. Training loss: 0.00016373821176625982
Epoch 19. Training loss: 0.0001612953169855869
Epoch 20. Training loss: 0.00015794030940498004
Epoch 21. Trai